<a href="https://colab.research.google.com/github/AntonioBorz/NeuroEmotions/blob/main/NeuroEmotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **НейроЭмоции**

Команда проекта (MIFIML):
1. Бикбулатова Айгуль Ришатовна.
2. Борзунов Антон Андреевич.
3. Булахов Юрий Эдуардович.
4. Ворошнина Анна Олеговна.
5. Голунов Артем Сергеевич.
6. Ситёв Роман Рустамович.
7. Чунарев Дмитрий Дмитриевич.

При реализации проекта использована предобученная модель машинного обучения - модель глубокого обучения, использующая рекуррентные нейронные сети (LSTM). Модель была разработана с помощью языка Python 3 и библиотеки PyTorch для анализа эмоциональной окраски коротких текстов из соцсети, написанных на русском языке, с помощью нейронной сети LSTM.

https://github.com/kategimranova/tonal_russian_texts

# **Направление «Умный городской гид»**

### Предобученная модель для анализа тональности текста может быть использована для анализа отзывов посетителей музеев и других культурных объектов для улучшения опыта посещения музеев и совершенствования предлагаемых услуг.



## Загрузка файлов с отзывами с Яндекс Карт о музеях РФ

*Загрузка файла отзывов с оценкой "4" и "5"*

In [ ]:
from google.colab import files
uploaded_pos = files.upload()

Saving 5_4.csv to 5_4.csv


*Загрузка файла отзывов с оценкой "1" и "2"*

In [ ]:
uploaded_neg = files.upload()

Saving 1_2.csv to 1_2.csv


## Загрузка файлов с комментариями из первоначальной модели

In [ ]:
uploaded_pos2 = files.upload()

Saving positive.csv to positive.csv


In [ ]:
uploaded_neg2 = files.upload()

Saving negative.csv to negative.csv


# Импорт необходимых библиотек

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import seaborn as sns
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

# Считывание данных

In [ ]:
n1 = ['ID', 'Rating', 'Date', 'Place Name', 'Text']
data_positive = pd.read_csv('5_4.csv')
data_negative = pd.read_csv('1_2.csv')

In [ ]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive2 = pd.read_csv('positive.csv', sep=';', names=n, usecols=['text'])
data_negative2 = pd.read_csv('negative.csv', sep=';', names=n, usecols=['text'])

*Анализируем датасет негативных отзывов*

In [ ]:
data_negative.head()

,ID,Rating,Date,Place Name,Text
0,9aa165746e5fe7680b6ecca490121348,1.0,2024-12-14T07:46:08.848Z,NaN,"Сплошное холуйство, ни в одном музее мира нет ..."
1,5ed68005412e49c94368d79b4f66b609,1.0,2024-12-07T19:26:15.582Z,NaN,Превратили музей в помойку. Я коренная Ленинг...
2,b8bbd3fa86a2c6676db41796b104f8d6,1.0,2024-12-04T14:02:55.583Z,NaN,музей для приезжих.
3,ab755f2b0aba4b50875689b990a95cff,1.0,2024-11-30T14:45:28.898Z,NaN,Мне не понравилось. Персонал грубый
4,7da0d6afaa3815914ee449b6c450fb86,1.0,2024-11-29T22:59:58.834Z,NaN,"Про шедевры не буду говорить, они – бесценны. ..."


In [ ]:
data_negative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3930 entries, 0 to 3929
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          3930 non-null   object 
 1   Rating      3930 non-null   float64
 2   Date        3928 non-null   object 
 3   Place Name  2191 non-null   object 
 4   Text        3928 non-null   object 
dtypes: float64(1), object(4)
memory usage: 153.6+ KB


In [ ]:
data_negative.describe()

,Rating
count,3930.000000
mean,1.382443
std,0.486046
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,2.000000


In [ ]:
data_negative['Place Name'].value_counts()

,count
Place Name,
Музей эмоций,195
Музей Гараж,90
Московский государственный музей С.А. Есенина,53
Робостанция,52
Друзья Русского музея,50
...,...
Ретрогараж,1
Музей Боевой Славы имени Героя Советского Союза Маршала Георгия Константиновича Жукова,1
Естественнонаучный музей,1


**Выводы:**

"Негативный" датасет содержит 3930 отзывов, больше всего отрицательных отзывов про "Музей эмоций". Средняя оценка - 1.38.

*Анализируем датасет положительных отзывов*

In [ ]:
data_positive.head()

,ID,Rating,Date,Place Name,Text
0,4fdd766b138a4e0dac65a800a8e0a29e,5.0,2024-07-15T17:11:06.182Z,Крейсер Аврора,Исторический памятник и одна из главных достоп...
1,b7f129bacaf6641f0765ab526fb58bcc,5.0,2024-11-04T08:57:33.544Z,Крейсер Аврора,Отлично в любую погоду. Неоднократно посещали ...
2,7e0001960c4be2045a279254a5684844,5.0,2024-11-28T12:52:02.036Z,Крейсер Аврора,"Интереснейшее место , атмосферное, и сам крейс..."
3,0ef09b957eb2b27b5a70924a0a81cfa9,5.0,2024-09-06T16:51:53.410Z,Крейсер Аврора,"Крейсер ""Аврора"" — это не только значимый исто..."
4,027a1e2197b2eceb8f46bb703dfa4c1a,5.0,2024-07-24T04:00:32.103Z,Крейсер Аврора,"Крейсер очень в хорошем состоянии, следят, мол..."


In [ ]:
data_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10077 entries, 0 to 10076
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          10077 non-null  object 
 1   Rating      10077 non-null  float64
 2   Date        10077 non-null  object 
 3   Place Name  10077 non-null  object 
 4   Text        10077 non-null  object 
dtypes: float64(1), object(4)
memory usage: 393.8+ KB


In [ ]:
data_positive['Place Name'].value_counts()

,count
Place Name,
Пропавшие в кинохронике,591
Музей Фаберже,590
Музей микроминиатюры Русский Левша Международной Гильдии Мастеров,588
Casino Museum,584
Юсуповский дворец на Мойке,584
Музей Анны Ахматовой,580
Фантазии и сны Зигмунда Фрейда,576
Аничков дворец,572
Российский этнографический музей,570


In [ ]:
data_positive.describe()

,Rating
count,10077.000000
mean,4.930138
std,0.254927
min,4.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


**Выводы:**

"Позитивный" датасет содержит 10077 отзывов, что говорит о значительной несбалансированности отрицательных и положительных отзывов. Средняя оценка 4.9. Наиболее оцениваемый объект - Пропавшие в кинохронике.

# Формирование сбалансированного датасета

In [ ]:
sample_size1 = 3000
reviews_withoutshuffle_1 = np.concatenate((
    data_positive['Text'].values[:sample_size1],
    data_negative['Text'].values[:sample_size1]
), axis=0)

labels_withoutshuffle_1 = np.asarray([1] * sample_size1 + [0] * sample_size1)

assert len(reviews_withoutshuffle_1) == len(labels_withoutshuffle_1)

sample_size2 = 40000
reviews_withoutshuffle_2 = np.concatenate((
    data_positive2['text'].values[:sample_size2],
    data_negative2['text'].values[:sample_size2]
), axis=0)

labels_withoutshuffle_2 = np.asarray([1] * sample_size2 + [0] * sample_size2)

assert len(reviews_withoutshuffle_2) == len(labels_withoutshuffle_2)

all_reviews = np.concatenate((reviews_withoutshuffle_1, reviews_withoutshuffle_2), axis=0)
all_labels = np.concatenate((labels_withoutshuffle_1, labels_withoutshuffle_2), axis=0)

reviews, labels = shuffle(all_reviews, all_labels, random_state=0)

assert len(reviews) == len(labels)

print(f"Количество отзывов: {len(all_reviews)}")

Количество отзывов: 86000


In [ ]:
print(all_reviews[1000])

Потрясающий вид снаружи и запустение внутри. После смерти Павла, его сын Александр все вывез из дворца. Потом тут было военное училище, в советское время - бюджетная организация. Интерьеры только начали восстанавливать. Ожидать такого великолепия, как в Павловске или Пушкине не стоит.


# Токенизация

In [ ]:
def tokenize():
  punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
  all_reviews = 'separator'.join(reviews)
  all_reviews = all_reviews.lower()
  all_text = ''.join([c for c in all_reviews if c not in punctuation])
  texts_split = all_text.split('separator')
  all_text = ' '.join(texts_split)
  words = all_text.split()
  return words, texts_split

words, texts_split = tokenize()

In [ ]:
new_reviews = []
for review in texts_split:
    review = review.split()
    new_text = []
    for word in review:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_reviews.append(new_text)

In [ ]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii - 1 for ii, word in enumerate(vocab, 1)}
reviews_ints = []
for review in new_reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])


In [ ]:
def add_pads(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i, row in enumerate(reviews_ints):
        if len(row) == 0:
            continue
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

# Разделение на обучающую, валидационную и тестовую выборки

In [ ]:
features = add_pads(reviews_ints, seq_length=30)
split_frac = 0.8 # 80% на обучающую выборку

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = labels[:split_idx], labels[split_idx:]
test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

# Определение режима: GPU или CPU

In [ ]:
train_gpu=torch.cuda.is_available()

print(train_gpu)

True


# Архитектура нейронной сети

In [ ]:
import torch.nn as nn

class LSTM_architecture(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, number_of_layers, drop=0.5):
        super(LSTM_architecture, self).__init__()
        self.output_size = output_size
        self.number_of_layers = number_of_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, number_of_layers,dropout=drop, batch_first=True)
        self.dropout = nn.Dropout(0.45)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, hidden_state):
        lstm_out, hidden_state = self.lstm(self.embedding(x.long()), hidden_state)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sigmoid(out)
        batch_size = x.size(0)
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        return sig_out, hidden_state

    def init_hidden_state(self, batch_size):
        weight = next(self.parameters()).data
        hidden_state = (weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_())
        return hidden_state


#Выбор гиперпараметров и инициализация сети

In [ ]:
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 100
hidden_dim = 128
n_layers = 2
model = LSTM_architecture(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение модели

In [ ]:
epochs = 4 #оптимальное количество эпох для того, чтобы модель достаточно обучилась, но не переобучилась
counter = 0
batch_num = 100
clip=5
if(train_gpu):
    model.cuda()
num_correct = 0
model.train()
for e in range(epochs):
    h = model.init_hidden_state(batch_size)
    for inputs, labels in train_loader:
        num_correct = 0
        counter += 1
        if(train_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        h = tuple([each.data for each in h])
        model.zero_grad()
        output, h = model.forward(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        if counter % batch_num == 0:
            val_h = model.init_hidden_state(batch_size)
            val_losses = []
            model.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if(train_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())
                val_losses.append(val_loss.item())

                #accuracy
                pred = torch.round(output.squeeze())
                correct_tensor = pred.eq(labels.float().view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if not train_gpu else np.squeeze(correct_tensor.cpu().numpy())
                num_correct += np.sum(correct)
                valid_acc = num_correct/len(valid_loader.dataset)

            model.train()
            print("Epoch: {} ;".format(e+1),
                  "Batch Number: {};".format(counter),
                  "Train Loss: {:.4f} ;".format(loss.item()),
                  "Valid Loss: {:.4f} ;".format(np.mean(val_losses)),
                  "Valid Accuracy: {:.4f}".format(valid_acc))


Epoch: 1 ; Batch Number: 100; Train Loss: 0.6949 ; Valid Loss: 0.6621 ; Valid Accuracy: 0.5966
Epoch: 1 ; Batch Number: 200; Train Loss: 0.6440 ; Valid Loss: 0.6537 ; Valid Accuracy: 0.6120
Epoch: 1 ; Batch Number: 300; Train Loss: 0.7453 ; Valid Loss: 0.6336 ; Valid Accuracy: 0.6226
Epoch: 1 ; Batch Number: 400; Train Loss: 0.6518 ; Valid Loss: 0.6174 ; Valid Accuracy: 0.6407
Epoch: 1 ; Batch Number: 500; Train Loss: 0.6093 ; Valid Loss: 0.6100 ; Valid Accuracy: 0.6527
Epoch: 1 ; Batch Number: 600; Train Loss: 0.5810 ; Valid Loss: 0.5978 ; Valid Accuracy: 0.6653
Epoch: 1 ; Batch Number: 700; Train Loss: 0.5398 ; Valid Loss: 0.5899 ; Valid Accuracy: 0.6766
Epoch: 1 ; Batch Number: 800; Train Loss: 0.6075 ; Valid Loss: 0.5873 ; Valid Accuracy: 0.6792
Epoch: 1 ; Batch Number: 900; Train Loss: 0.6289 ; Valid Loss: 0.5828 ; Valid Accuracy: 0.6845
Epoch: 1 ; Batch Number: 1000; Train Loss: 0.5986 ; Valid Loss: 0.5713 ; Valid Accuracy: 0.6903
Epoch: 1 ; Batch Number: 1100; Train Loss: 0.6037

# Предсказание окраски по тексту

In [ ]:
def tokenize_text(test_review):
    test_review = test_review.lower()
    punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()

    new_text = []
    for word in test_words:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)

    test_ints = []
    # Пропускаем слова, отсутствующие в словаре
    test_ints.append([vocab_to_int[word] for word in new_text if word in vocab_to_int])

    return test_ints

In [ ]:
def predict(model, test_review, sequence_length=30):
    model.eval()
    test_ints = tokenize_text(test_review)
    seq_length = sequence_length

    if len(test_ints[0]) == 0:
        print("Не удалось распознать слова из отзыва.")
        return

    features = add_pads(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)
    batch_size = feature_tensor.size(0)
    h = model.init_hidden_state(batch_size)
    if (train_gpu):
        feature_tensor = feature_tensor.cuda()
    output, h = model(feature_tensor, h)

    pred = torch.round(output.squeeze())
    print('Вероятность положительного ответа {:.6f}'.format(output.item()))

    if (pred.item() == 1):
        print("Положительный отзыв")
    else:
        print("Негативный отзыв")

# Тест модели

In [ ]:
seq_length = 30
test_review = "Классное место и очень познавательно. Приходить с детьми или компанией для расширения кругозора. Если платные услуги и ты содешь почувствовать себя пилотом 😀 рекомендую для посещения"
predict(model, test_review, seq_length)

Вероятность положительного ответа 0.997863
Положительный отзыв
